In [ ]:
%matplotlib widget
from util import get_path
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from extract_graph import generate_nx_graph, transform_list, generate_skeleton, generate_nx_graph_from_skeleton, from_connection_tab
from node_id import reconnect_degree_2
import ast
from plotutil import plot_t_tp1, compress_skeleton
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign
from datetime import datetime,timedelta
from node_id import orient
import pickle
from matplotlib.widgets import CheckButtons
import scipy.io as sio
import imageio
from pymatreader import read_mat
import os
from matplotlib import colors
from random import choice
from experiment_class_surf import Experiment,clean_exp_with_hyphaes
from hyphae_id import clean_and_relabel, get_mother, save_hyphaes, resolve_ambiguity_two_ends, solve_degree4, clean_obvious_fake_tips
from extract_graph import prune_graph
from skimage.measure import profile_line
import math
from Analysis.util import *
from directory import directory, path_code
from Analysis.data_info import *
from extract_width import *

In [ ]:
directory = "/projects/0/einf914/data/"

In [ ]:
exp = get_exp((40,0,37),directory)
# exp_copy = Experiment(40,directory)

In [41]:
t = 0
edge_test = get_width_info(exp,t)

149
(14, 20)
34.060956030904535
(21, 20)
20.412199102785838
(20, 19)
23.92461937487765
(20, 67)
34.485723746252184
(6, 15)
35.02825022321903
(15, 16)
0.0
(15, 25)
35.09983307251886
(147, 129)
12.255486697543368
(129, 125)
20.39304202002261
(129, 130)
44.47933884297521
(25, 24)
0.0
(25, 32)
37.40697138588558
(10, 8)
23.795302378371893
(8, 7)
23.81865618129266
(8, 9)
27.33861612625362
(9, 4)
31.42348013319164
(9, 33)
36.280194262986534
(126, 125)
0.0
(128, 125)
42.45630957433768
(125, 122)
44.45771168426942
(120, 122)
42.68964016747695
(122, 107)
35.64104370769046
(122, 119)
30.67554932201258
(4, 3)
30.55476323743468
(4, 5)
0.0
(148, 144)
45.929034613434204
(144, 145)
31.470693410223923
(144, 141)
47.74492100120772
(107, 84)
39.46216028488071
(107, 111)
23.934607387854456
(119, 111)
30.670085737803262
(119, 113)
34.86472164808603
(84, 71)
36.51026019535725
(84, 80)
30.649328994747783
(111, 108)
23.905464018887734
(71, 67)
37.338236265199534
(71, 68)
29.02198832469729
(67, 68)
44.30123495

In [3]:
# exp_copy.copy(exp)
# labels = {node for g in exp_copy.nx_graph for node in g}
# exp_copy.nodes=[]
# for label in labels:
#     exp_copy.nodes.append(Node(label,exp_copy))

In [61]:
plt.close('all')
edge_exp.end.show_source_image(10,10)
edge_exp.begin.show_source_image(10,10)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
edge = choice(list(exp.nx_graph[10].edges))
edge_exp = Edge(Node(edge[0],exp),Node(edge[1],exp),exp)
# edge_exp = Edge(Node(366,exp),Node(188,exp),exp)

In [ ]:
plt.close('all')

get_width_edge(edge_exp,50,10,False,threshold_averaging  = 5)

In [4]:
def get_source_image(experiment,pos,t,local,force_selection = None):
    x,y=pos[0],pos[1]
    ims,posimg=experiment.find_image_pos(x,y,t,local)
    if force_selection is None:
        dist_border=[min([posimg[1][i],3000-posimg[1][i],posimg[0][i],4096-posimg[0][i]]) for i in range(posimg[0].shape[0])]
        j=np.argmax(dist_border)
    else:
        dist_last=[np.linalg.norm(np.array((posimg[1][i],posimg[0][i])) - np.array(force_selection)) for i in range(posimg[0].shape[0])]
        j=np.argmin(dist_last)
    return(ims[j],(posimg[1][j],posimg[0][j]))

In [76]:
def get_width_pixel(edge,index,im,pivot,before,after,t,size = 20,width_factor = 60,averaging_size = 100,threshold_averaging = 10):
    imtab=im
#     print(imtab.shape)
#     print(int(max(0,pivot[0]-averaging_size)),int(pivot[0]+averaging_size))
    threshold = np.mean(imtab[int(max(0,pivot[0]-averaging_size)):int(pivot[0]+averaging_size),int(max(0,pivot[1]-averaging_size)):int(pivot[1]+averaging_size)]-threshold_averaging)
    orientation=np.array(before)-np.array(after)
    perpendicular = [1,-orientation[0]/orientation[1]] if orientation[1]!=0 else [0,1]
    perpendicular_norm=np.array(perpendicular)/np.sqrt(perpendicular[0]**2+perpendicular[1]**2)
    point1=np.around(np.array(pivot)+width_factor*perpendicular_norm)
    point2=np.around(np.array(pivot)-width_factor*perpendicular_norm)
    point1=point1.astype(int)
    point2=point2.astype(int)
#     p = profile_line(imtab, point1, point2,mode='constant')
#     fig = plt.figure()
#     ax = fig.add_subplot(111)
#     ax.plot(p)
#     derivative = [p[i+1]-p[i] for i in range(len(p)-1)]
#     fig = plt.figure()
#     ax = fig.add_subplot(111)
#     ax.plot([np.mean(derivative[5*i:5*i+5]) for i in range(len(derivative)//5)])
    problem=False
    arg = len(p)//2
    if p[arg]>threshold:
        arg = np.argmin(p)
#     we_plot=randrange(1000)
    while  p[arg]<=threshold:
        if arg<=0:
#             we_plot=50
            problem=True
            break
        arg-=1
    begin = arg
    arg = len(p)//2
    if p[arg]>threshold:
        arg = np.argmin(p)
    while  p[arg]<=threshold:
        if arg>=len(p)-1:
#             we_plot=50
            problem=True
            break
        arg+=1
    end = arg
#     print(end-begin,len(p))
    return(np.linalg.norm(point1-point2)*(end-begin)/len(p))

In [79]:
def get_width_edge(edge,resolution,t,local=False, threshold_averaging = 10):
    pixel_conversion_factor = 1.725
    pixel_list = edge.pixel_list(t)
    pixels = []
    indexes = []
    source_images = []
    poss = []
    widths={}
    if len(pixel_list)>3*resolution:
        for i in range(0,len(pixel_list)//resolution):
            index = i*resolution
            indexes.append(index)
            pixel = pixel_list[index]
            pixels.append(pixel)
            source_img,pos = get_source_image(edge.experiment,pixel,t,local)
            source_images.append(source_img)
            poss.append(pos)
    else:
        indexes = [0,len(pixel_list)//2,len(pixel_list)-1]
        for index in indexes:
            pixel = pixel_list[index]
            pixels.append(pixel)
            source_img,pos = get_source_image(edge.experiment,pixel,t,local)
            source_images.append(source_img)
            poss.append(pos)
#     print(indexes)
    for i, index in enumerate(indexes[1:-1]):
        source_img = source_images[i+1]
        pivot = poss[i+1]
        _,before = get_source_image(edge.experiment,pixels[i],t,local,pivot)
        _,after = get_source_image(edge.experiment,pixels[i+2],t,local,pivot)
        plot_t_tp1([0,1,2],[],{0 : pivot,1 : before, 2 : after},None,source_img,source_img)
        width = get_width_pixel(edge,index,source_img,pivot,before,after,t,threshold_averaging = threshold_averaging)
#         print(width*pixel_conversion_factor)
        widths[pixel_list[index]]=width*pixel_conversion_factor
    edge.experiment.nx_graph[t].get_edge_data(edge.begin.label,edge.end.label)['width'] = widths
    return(widths)      

In [39]:
def get_width_info(experiment,t,resolution = 50):
    edge_width={}
    graph = experiment.nx_graph[t]
#     print(len(list(graph.edges)))
    for edge in graph.edges:
#         print(edge)
        edge_exp = Edge(Node(edge[0],experiment),Node(edge[1],experiment),experiment)
        mean = np.mean(list(get_width_edge(edge_exp,resolution,t).values()))
        print(np.mean(list(get_width_edge(edge_exp,resolution,t).values())))
        edge_width[edge]=mean
    return(edge_width)

In [55]:
max_width = np.max(list(edge_test.values()))
max_width

206.5682698404925

In [58]:
plt.cm.get_cmap('plasma')([width/max_width for width in edge_test.values()])

array([[3.62553e-01, 3.24300e-03, 6.49245e-01, 1.00000e+00],
       [2.54627e-01, 1.38820e-02, 6.15419e-01, 1.00000e+00],
       [2.80648e-01, 1.14880e-02, 6.25038e-01, 1.00000e+00],
       [3.62553e-01, 3.24300e-03, 6.49245e-01, 1.00000e+00],
       [3.68733e-01, 2.72400e-03, 6.50601e-01, 1.00000e+00],
       [5.03830e-02, 2.98030e-02, 5.27975e-01, 1.00000e+00],
       [3.68733e-01, 2.72400e-03, 6.50601e-01, 1.00000e+00],
       [1.86213e-01, 1.88030e-02, 5.87228e-01, 1.00000e+00],
       [2.54627e-01, 1.38820e-02, 6.15419e-01, 1.00000e+00],
       [4.41732e-01, 1.54000e-03, 6.60069e-01, 1.00000e+00],
       [5.03830e-02, 2.98030e-02, 5.27975e-01, 1.00000e+00],
       [3.87183e-01, 1.43400e-03, 6.54177e-01, 1.00000e+00],
       [2.80648e-01, 1.14880e-02, 6.25038e-01, 1.00000e+00],
       [2.80648e-01, 1.14880e-02, 6.25038e-01, 1.00000e+00],
       [3.06210e-01, 8.90200e-03, 6.33694e-01, 1.00000e+00],
       [3.37683e-01, 5.61800e-03, 6.43049e-01, 1.00000e+00],
       [3.74897e-01, 2.2

In [74]:
plt.close('all')
t=0
fig = plt.figure()
ax = fig.add_subplot(111)
graph = exp.nx_graph[t]
for edge in graph.edges:
    edge_exp = Edge(Node(edge[0],exp),Node(edge[1],exp),exp)
    begin = edge_exp.begin.pos(t)
    end = edge_exp.end.pos(t)
    ax.plot((begin[1],end[1]), (begin[0],end[0]),color = plt.cm.get_cmap('plasma')(edge_test[edge]/60))
# plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [75]:
exp.plot_raw(0)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
edge_test.experiment.nx_graph[t].get_edge_data(edge_test.begin.label,edge_test.end.label)

In [323]:
plt.close('all')
get_width_edge(edge_test,50,4,True,threshold_averaging  = 5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(3000, 4096)
1711 1911


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

24 143.053775


{(8717, 7032): 40.86386931802906}

In [224]:
exp_2.plot([4],[[edge_test.end.label,edge_test.begin.label]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
# node = choice(exp_2.nodes)
# node.ts()

In [139]:
edge_test

Edge(1679,1823)

In [138]:
edge_test = node.neighbours(4)[0].edges(4)[0].end.edges(4)[0]